# The Order of Finite Differencing

This notebook demonstrates the difference between 2nd and 4th order finite differencing methods when computing the stress-energy tensor. The `diff_order` parameter in `get_energy_tensor()` allows you to control the accuracy of the numerical derivatives.

## Background

When computing derivatives numerically, higher-order methods provide more accurate results but require more grid points. For energy tensor calculations:
- **4th order** (default): More accurate, requires minimum 5 grid points in each direction
- **2nd order**: Less accurate, requires minimum 3 grid points, useful for checking finite differencing errors with sharp metric transitions

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from warpfactory.metrics.alcubierre import get_alcubierre_metric
from warpfactory.solver.energy import get_energy_tensor

## Create Alcubierre Metric

We'll create an Alcubierre warp drive metric to compare the two finite differencing methods.

In [ ]:
# Set up grid parameters
grid_size = [1, 20, 20, 20]
world_center = [(grid_size[i] + 1) / 2 for i in range(4)]
velocity = 0.5
R = 5
sigma = 0.5

# Create the Alcubierre metric
metric = get_alcubierre_metric(
    grid_size=grid_size,
    world_center=world_center,
    velocity=velocity,
    radius=R,
    sigma=sigma
)

print(f"Metric: {metric.name}")
print(f"Grid size: {grid_size}")
print(f"Velocity: {velocity}c")
print(f"Radius: {R}")
print(f"Sigma: {sigma}")

## Compute Energy Tensors with Different Orders

Now we'll compute the energy tensor using both 4th order (default) and 2nd order finite differencing.

In [ ]:
# Compute energy tensor using fourth order (default)
energy_tensor_fourth = get_energy_tensor(metric, diff_order=4)

# Compute energy tensor using second order
energy_tensor_second = get_energy_tensor(metric, diff_order=2)

print("Fourth order energy tensor computed")
print("Second order energy tensor computed")

## Visualize Fourth Order Results

Let's visualize all components of the energy tensor computed with 4th order finite differencing.

In [ ]:
# Plotting Energy Tensor - Fourth Order
fig, axes = plt.subplots(4, 4, figsize=(16, 16))
fig.suptitle(f'{metric.name} Energy Tensor - Fourth Order', fontsize=16)

t_slice = 0
z_slice = int(world_center[3])

for i in range(4):
    for j in range(4):
        ax = axes[i, j]
        data = energy_tensor_fourth.tensor[(i, j)][t_slice, :, :, z_slice]
        
        im = ax.imshow(data.T, origin='lower', cmap='RdBu_r', aspect='auto')
        ax.set_title(f'T[{i},{j}]')
        ax.set_xlabel('x')
        ax.set_ylabel('y')
        plt.colorbar(im, ax=ax)

plt.tight_layout()
plt.show()

## Visualize Second Order Results

Now let's visualize the same components computed with 2nd order finite differencing.

In [ ]:
# Plotting Energy Tensor - Second Order
fig, axes = plt.subplots(4, 4, figsize=(16, 16))
fig.suptitle(f'{metric.name} Energy Tensor - Second Order', fontsize=16)

for i in range(4):
    for j in range(4):
        ax = axes[i, j]
        data = energy_tensor_second.tensor[(i, j)][t_slice, :, :, z_slice]
        
        im = ax.imshow(data.T, origin='lower', cmap='RdBu_r', aspect='auto')
        ax.set_title(f'T[{i},{j}]')
        ax.set_xlabel('x')
        ax.set_ylabel('y')
        plt.colorbar(im, ax=ax)

plt.tight_layout()
plt.show()

## Compare the Difference

To better understand the impact of the finite differencing order, let's compute and visualize the difference between the two methods.

In [ ]:
# Plotting Difference Between Fourth and Second Order
fig, axes = plt.subplots(4, 4, figsize=(16, 16))
fig.suptitle('Difference: Fourth Order - Second Order', fontsize=16)

for i in range(4):
    for j in range(4):
        ax = axes[i, j]
        data_fourth = energy_tensor_fourth.tensor[(i, j)][t_slice, :, :, z_slice]
        data_second = energy_tensor_second.tensor[(i, j)][t_slice, :, :, z_slice]
        difference = data_fourth - data_second
        
        im = ax.imshow(difference.T, origin='lower', cmap='RdBu_r', aspect='auto')
        ax.set_title(f'ΔT[{i},{j}]')
        ax.set_xlabel('x')
        ax.set_ylabel('y')
        plt.colorbar(im, ax=ax)

plt.tight_layout()
plt.show()

## Quantitative Comparison

Let's examine the numerical differences between the two methods.

In [ ]:
# Calculate statistics for each component
print("Comparison of Fourth vs Second Order Finite Differencing")
print("=" * 60)
print(f"{'Component':<12} {'Max Diff':<15} {'RMS Diff':<15}")
print("-" * 60)

for i in range(4):
    for j in range(4):
        data_fourth = energy_tensor_fourth.tensor[(i, j)][t_slice, :, :, z_slice]
        data_second = energy_tensor_second.tensor[(i, j)][t_slice, :, :, z_slice]
        difference = data_fourth - data_second
        
        max_diff = np.max(np.abs(difference))
        rms_diff = np.sqrt(np.mean(difference**2))
        
        print(f"T[{i},{j}]        {max_diff:<15.6e} {rms_diff:<15.6e}")

print("=" * 60)

## Cross-Section Comparison

Let's examine a 1D cross-section through the center to see the differences more clearly.

In [ ]:
# Plot cross-sections for key components
fig, axes = plt.subplots(2, 2, figsize=(14, 10))
fig.suptitle('Cross-Section Comparison: Fourth vs Second Order', fontsize=16)

# Key components to examine
components = [(0, 0), (0, 1), (1, 1), (2, 2)]
y_center = int(world_center[2])

for idx, (i, j) in enumerate(components):
    ax = axes[idx // 2, idx % 2]
    
    data_fourth = energy_tensor_fourth.tensor[(i, j)][t_slice, :, y_center, z_slice]
    data_second = energy_tensor_second.tensor[(i, j)][t_slice, :, y_center, z_slice]
    
    ax.plot(data_fourth, 'b-', label='4th Order', linewidth=2)
    ax.plot(data_second, 'r--', label='2nd Order', linewidth=2)
    ax.set_title(f'T[{i},{j}] Cross-Section')
    ax.set_xlabel('x index')
    ax.set_ylabel(f'T[{i},{j}]')
    ax.legend()
    ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## Deciding Between Fourth vs. Second Order

**Recommendation:** For almost all applications, **4th order is preferred** over 2nd order because:
- Higher accuracy for smooth metrics
- Better convergence properties
- Standard in most computational physics applications

**When to use 2nd order:**
- When analyzing sharp metric transitions (discontinuities or near-discontinuities)
- As a diagnostic tool to check finite differencing errors
- When computational resources are extremely limited (though this is rarely an issue for typical grid sizes)

## Summary

In this notebook, we:
1. Created an Alcubierre warp drive metric
2. Computed the stress-energy tensor using both 4th and 2nd order finite differencing
3. Visualized all components of both energy tensors
4. Compared the differences between the two methods
5. Analyzed cross-sections to understand where the methods differ

The `diff_order` parameter in `get_energy_tensor()` provides flexibility for different analysis needs, with 4th order being the recommended default for most applications.